In [54]:
from interpret import interpret
from querry import *
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
from DataSet import DataSet

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense
from keras import optimizers
from keras.metrics import categorical_crossentropy
from keras.wrappers.scikit_learn import KerasRegressor

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
seed = 8888

In [ ]:
# just for testing, at the end.
def test_data(compound, T):
    """Converts one sample to raw data for predicting thru ANN"""
    array = compound_short_descriptors(compound) + [T]
    array = X_train_scaler.transform(X_train_pn)
    return np.array(array)

In [3]:
# create a new DataSet object
dataset = DataSet()
dataset.get_data('../data/TE_survey_csv_repaired.csv')

dataset.clean()
dataset.drop(['Authors', 'DOI', 'Comments', 'Comments.1', 'Author of Unit Cell','Unit Cell DOI'])

# use extrapolate_400K to extrapolate more row data
dataset_2 = DataSet()
dataset_2.data = dataset.extrapolate_400K([])
dataset_2.get_info()

1450 rows and 4 columns.
Components are: 
Formula, Resist, Seebeck, T (K)


In [4]:
# make an array containing the atomic descriptors
array = [compound_short_descriptors(x) for x in dataset_2.df['Formula'].values]
ndf = pd.DataFrame.from_records(array)
ndf = ndf.join(dataset_2.df[['T (K)']])
ndf = ndf.fillna(0)
print(ndf) # looks good!

           0     1             2       3      4       5        6      7  \
0     1.0000  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
1     1.0000  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
2     1.0000  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
3     1.0000  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
4     0.9800  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
5     0.9800  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
6     0.9800  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
7     0.9800  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
8     0.9600  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
9     0.9600  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
10    0.9600  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
11    0.9600  16.0  3.400000e-08  3810.0  200.0  1757.0  1115.00   17.0   
12    0.9400  16.0  3.400

In [5]:
X = ndf.values
Y = dataset_2.df[['Seebeck']].values

In [6]:
X_train_pn, X_test_pn, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.20,
                                                    random_state=seed)

# create the scaler from the training data only and keep it for later use
X_train_scaler = StandardScaler().fit(X_train_pn)
# apply the scaler transform to the training data
X_train = X_train_scaler.transform(X_train_pn)
# tranform the testing set
X_test = X_train_scaler.transform(X_test_pn)


In [83]:
# make the neurial nets
model = Sequential([
    Dense(100, input_shape=(81,), activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(1, activation='relu')
])


In [84]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_59 (Dense)             (None, 100)               8200      
_________________________________________________________________
dense_60 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_61 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_62 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_63 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_64 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_65 (Dense)             (None, 100)               10100     
__________

In [89]:
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mean_squared_error'])

In [90]:
model.fit(X_train, y_train, batch_size=10, epochs=10, shuffle=True, verbose=2)

Epoch 1/10
 - 1s - loss: 152.7692 - mean_squared_error: 36728.7407
Epoch 2/10
 - 0s - loss: 152.7692 - mean_squared_error: 36728.7407
Epoch 3/10
 - 0s - loss: 152.7692 - mean_squared_error: 36728.7409
Epoch 4/10
 - 0s - loss: 152.7692 - mean_squared_error: 36728.7405
Epoch 5/10
 - 0s - loss: 152.7692 - mean_squared_error: 36728.7405
Epoch 6/10
 - 0s - loss: 152.7692 - mean_squared_error: 36728.7407
Epoch 7/10
 - 0s - loss: 152.7692 - mean_squared_error: 36728.7405
Epoch 8/10
 - 0s - loss: 152.7692 - mean_squared_error: 36728.7404
Epoch 9/10
 - 0s - loss: 152.7692 - mean_squared_error: 36728.7404
Epoch 10/10
 - 0s - loss: 152.7692 - mean_squared_error: 36728.7407


FAILURE.